<a href="https://colab.research.google.com/github/astroviki/TrafficAnalysis/blob/main/notebooks/OrientDBQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* Snížit dimenzi analyzovaných dat s ohledem na známe cílové adresy (seznam českých a mezinárodních adres)
* Přidat atribut související s hlavičkama paketů.

* **Detekce stejné sekvence spojení v rámci síťového provozu**
* Detekce Časové korelace mezi nody (najít a ukázat příklad)
* Detekce anomálního přerušení spojení (najít a ukázat příklad)

# Detekce stejné sekvence v rámci síťového provozu
## OrientDB - grafová databáze

Data z poskytnutých CSV souborů jsou importována do databáze [OrientDB](https://orientdb.org/) (GPL grafová databáze). Výhodou je škálovatelnost, redukce dat, dotazování pomocí SQL jazyka. Prozatím na adrese livingscience.info:2480. Databáze **TestSmall* uživatel *root* a *6172admi*.

In [2]:
!pip install pyorientdb
!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from itertools import combinations
import pyorientdb
import pandas as pd
from datetime import datetime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.1 MB/s eta 0:00:00


## Hledání podobných vzorů
Selekce všech spojení ze zdrojové IP adresy 192.168.5.149 jejich četnost je větší než jedna. Metoda by měla najít v rámci této komunikace opakující se konektivitu na cílové adresy 192.168.100.100, 192.168.210.100, 192.168.89.70. Popřípadě i zachytit podobnou sekvenci.

**Plán**
- SQL dotaz na selekci spojení s četností větší než jedna. Výsledek uložit do *Pandas DataFrame*
-  Transformace IP adresy do textové reprezentace 6 znaků. Využijeme CRC32 hashovací funkce a *base_32* kódovací funkce.
- Sestavení textového řetězce odpovídající všem spojením jdoucím po sobě v čase s četností větší než jedna.
- Hledání stejných subřetězců
- Hledání podobnch subřetězců s využitím pravděpodobnostních diagramů

In [3]:
# Připojení k serveru OrientDB
client = pyorientdb.OrientDB("livingscience.info", 2424)  # Upravte IP adresu a port podle konfigurace
session_id = client.connect("root", "6172admi")  # Přihlášení jako root, zadejte správné heslo

# Otevření databáze (např. nettest)
client.db_open("TestSmall", "root", "6172admi")

query = f"""
SELECT
    @rid AS edge_id,
    in.address AS target_ip,
    in.unique_string AS target_string,
    out.address AS source_ip,
    out.unique_string AS source_string,
    port, ip_count, duration, time
FROM COMMUNICATES_WITH
ORDER BY time
LIMIT -1
"""

results=client.command(query)

# Transformace výsledků do pandas DataFrame
data = []
for record in results:
    data.append({
        "edge_id": record.oRecordData.get('edge_id'),
        "source_ip": record.oRecordData.get('source_ip'),
        "source_string": record.oRecordData.get('source_string'),
        "target_ip": record.oRecordData.get('target_ip'),
        "target_string": record.oRecordData.get('target_string'),
        "port": record.oRecordData.get('port'),
        "ip_count": record.oRecordData.get('ip_count'),
        "duration": float(record.oRecordData.get('duration'))/1.0e6,
        "time": float(record.oRecordData.get('time'))/1.0e6
    })

# Převést data na DataFrame pro snadné zobrazení
df = pd.DataFrame(data)

# Převod časového údaje v unix-formátu na čitelnější formát (rok-měsíc-den a čas)
#df['time'] = pd.to_datetime(df['time'], unit='s')
#df['duration']=df['duration']/60

# Vytvoření sloupce s četností IP adres
#df['ip count'] = df['target_ip'].map(df['target_ip'].value_counts())

Výsledný import dat z databáze *TestSmall* OrientDB (livingscience.info:2480)

### Transformace IP adresy na řetězec

Oproti použití IP adresy spatřuji následné výhody

*   Redukce počtu znaků
*   Vyhnutí se problémům z reprezentací IP adresy protokol IPv4 versus IPv6
*   Při hledání podobnosti slov opakující se segment sítě např 192.168. apod může vést ke zbytečnému a nechtěnému biasu



In [4]:
df

,edge_id,source_ip,source_string,target_ip,target_string,port,ip_count,duration,time
0,#26:13634,192.168.5.1,13rd6yr,192.168.5.90,k1k5wo,21,1774,0.166400,1.725314e+09
1,#27:13634,17.248.209.64,k3ypa9,192.168.5.165,vqiq6p,65156,153,0.000380,1.725314e+09
2,#28:13634,192.168.5.165,vqiq6p,17.248.209.64,k3ypa9,443,182,0.123411,1.725314e+09
3,#25:13635,192.168.5.133,1ry05lt,255.255.255.255,1421d6x,6667,9853,30026.079136,1.725314e+09
4,#26:13635,192.168.5.1,13rd6yr,192.168.5.90,k1k5wo,55600,1774,0.002661,1.725314e+09
...,...,...,...,...,...,...,...,...,...
93671,#25:13633,fe80:0:0:0:d221:f9ff:fe7b:8ec3,osqd8,ff02:0:0:0:0:0:0:1,i5hxaz,10001,9843,0.000000,1.725482e+09
93672,#26:13633,192.168.5.160,kfc7y,192.168.5.1,13rd6yr,53,29806,0.026462,1.725482e+09
93673,#27:13633,192.168.5.160,kfc7y,192.168.5.1,13rd6yr,53,29806,0.025711,1.725482e+09
93674,#28:13633,192.168.5.160,kfc7y,192.168.5.1,13rd6yr,53,29806,0.028743,1.725482e+09



### Filtrace dat
Nyní přistoupíme k filtraci dat, tak abychom eliminovali provozní šum. Z logiky věci se nabízí vyloučit z analýzy spojení

*   Spojení, u kterých je doba trvání nulová
*   Spojení na běžných portech (80,443,53)
*   Spojeni mDNS pro Apple Itunes 224.0.0.251 port 5353

Proces filtrace se ukazuje jako klíčový, musí být dostatečně robustní a modifikovatelný.



In [5]:
# Definování časového intervalu
start_time = 1725342700
end_time = 1725478592

# Seznam portů, které chcete vyloučit
excluded_ports = [80, 443, 53, 67, 68, 123, 993, 995, 5353]



# Filtrování řádků, které spadají do daného časového intervalu
filtered_df = df[(df['time'] >= start_time) & (df['time'] <= end_time) &
                 (df['ip_count'].astype(int) < 100) & (~df['port'].isin(excluded_ports))]
#filtered_df = df[(df['time'] >= start_time) & (df['time'] <= end_time

#type(int(df['ip_count'][0]))
filtered_df

,edge_id,source_ip,source_string,target_ip,target_string,port,ip_count,duration,time
35894,#28:22607,192.168.5.135,46pidw,192.168.89.70,fvdwjp,26941,6,0.000000,1.725343e+09
35896,#26:22608,192.168.5.135,46pidw,192.168.89.70,fvdwjp,30124,6,0.000000,1.725343e+09
35900,#26:22609,192.168.5.165,vqiq6p,157.240.30.55,ljs7tb,5222,29,0.001157,1.725343e+09
36002,#28:22634,192.168.5.165,vqiq6p,142.250.102.188,drdkli,5228,15,85.442515,1.725343e+09
36132,#26:22667,173.194.79.84,3sng90,192.168.5.157,1eujf9a,65306,56,0.001915,1.725343e+09
...,...,...,...,...,...,...,...,...,...
83815,#25:11169,142.250.27.108,1ndualh,192.168.5.140,dducrg,60497,62,2.668876,1.725478e+09
83816,#26:11169,142.250.27.108,1ndualh,192.168.5.140,dducrg,60498,62,3.204862,1.725478e+09
83916,#26:11194,85.207.218.179,1nx6jdz,192.168.5.157,1eujf9a,60056,56,0.305521,1.725478e+09
83917,#27:11194,77.75.78.99,1allvtg,192.168.5.157,1eujf9a,60053,56,30.434154,1.725478e+09


Dalším úkolem je vygenerovat řetězec, odpovídající po sobě jdoucím spojením.

In [6]:
# Spojení hodnot z obou sloupců do jednoho sloupce
#df['combined_string'] = df['source_string'] + df['target_string']

# Spojení všech hodnot do jednoho výsledného řetězce
raw_string = ' '.join(filtered_df['target_string'])

#words =raw_string.split(' ')

#for i in range(0, len(words), 14):
#    print(' '.join(words[i:i+14]))

def reduce_repeated_words(text):
    words = text.split()  # Rozdělení textu na slova
    reduced_words = [words[0]]  # První slovo vždy zahrneme

    # Projdeme slova a přidáme jen ta, která se nerovnají předchozímu
    for i in range(1, len(words)):
        if words[i] != words[i - 1]:
            reduced_words.append(words[i])

    return ' '.join(reduced_words)

final_string = reduce_repeated_words(raw_string)

words =final_string.split(' ')

for i in range(0, len(words), 14):
    print(' '.join(words[i:i+14]))


fvdwjp ljs7tb drdkli 1eujf9a mgvi0p drdkli 1eujf9a scmfa8 17elv0d wh5s7c 17elv0d 1pni6gg 357uj3 vj8pux
dducrg lg1yl5 dducrg lg1yl5 13ipmgh lg1yl5 d2ry8q lg1yl5 dducrg 649r85 lg1yl5 1s6cs9u lg1yl5 1eujf9a
lg1yl5 1oxvk27 lg1yl5 38yath 1vllswt 2iruzp ogi2do qqp5nh 1eujf9a 357uj3 15zv74q lg1yl5 1hlntbq lg1yl5
1hlntbq ogi2do 357uj3 lg1yl5 357uj3 972qos qqp5nh 1eujf9a 15zv74q d2ry8q dducrg 1g9ocqk 7kg4tt lg1yl5
mgvi0p lg1yl5 mgvi0p 357uj3 dducrg 357uj3 lg1yl5 7kg4tt 357uj3 lg1yl5 7kg4tt mgvi0p 1hoeid6 tp6l1q
1hoeid6 tp6l1q 1f2nsz1 lg1yl5 4hcnte nnlnhw 1p6tzse 1hoeid6 4hcnte 1f2nsz1 nnlnhw 1p6tzse nnlnhw 1p6tzse
4hcnte nnlnhw 1p6tzse nnlnhw 1p6tzse 7kg4tt lg1yl5 120l0x4 cufy2s wh5s7c cufy2s mgvi0p 7kg4tt lg1yl5
ljs7tb 1hoeid6 tp6l1q 1f2nsz1 1p6tzse tp6l1q 4hcnte 1f2nsz1 nnlnhw tp6l1q 1hoeid6 1f2nsz1 1hoeid6 1f2nsz1
1p6tzse tp6l1q lg1yl5 4hcnte nnlnhw tp6l1q 4hcnte nnlnhw tp6l1q lg1yl5 1eujf9a wh5s7c 1ejijds lg1yl5
1lajbrt dmyacq 7kg4tt ljs7tb lg1yl5 exk87h 1eujf9a dducrg mgvi0p cufy2s 1u6lbz 

### Textová reprezentace po sobě jdoucích spojení



*   192.168.100.100 1vllswt
*   192.168.210.100 2iruzp

13ipmgh d2ry8q lg1yl5 dducrg 649r85 1s6cs9u lg1yl5 lg1yl5 1oxvk27 38yath **1vllswt 2iruzp** qqp5nh 1eujf9a

8kd25p 1f7ty2e 1f7ty2e 53e4bk 53e4bk **2iruzp 1vllswt** ni6bt2 1kqzkkp 120l0x4 1kqzkkp 1wjkezq 53e4bk 1kqzkkp


### Metoda pro hledání opakující se sekvence (Upravit pro list s redukovaným textem)
Úkolem metody je najít v textové reprezentaci opakující se slova. V prvním přiblížení se soustředíme na hledání opakující se dvojice slov bez ohledu v jakém pořadí se opakují.

In [ ]:

def find_similar_pairs(df):
    similar_pairs = []
    seen_pairs = set()

    # Najděte všechny po sobě jdoucí dvojice, které nejsou stejné
    for i in range(len(df) - 1):
        str1 = df['target_string'].iloc[i]
        str2 = df['target_string'].iloc[i + 1]
        time1 = df['time'].iloc[i]
        time2 = df['time'].iloc[i + 1]

        # Zkontrolujte, že hodnoty nejsou stejné
        if str1 != str2:
            pair = (str1, str2)
            reversed_pair = (str2, str1)

            # Najděte podobné dvojice (opačné pořadí)
            if reversed_pair in seen_pairs:
                # Přidejte pouze, pokud podobná dvojice nebyla již přidána
                if reversed_pair not in similar_pairs:
                    similar_pairs.append(((str1, str2), (time1, time2)))
            else:
                seen_pairs.add(pair)

    # Výpis výsledků
    for pair, times in similar_pairs:
        print(f"Similar Pair: {pair}, Times: {times}")

# Ukázkový DataFrame pro testování


# Zavolání funkce
find_similar_pairs(filtered_df)


Similar Pair: ('dducrg', 'lg1yl5'), Times: (1725457923.709546, 1725457926.325872)
Similar Pair: ('lg1yl5', '357uj3'), Times: (1725458109.235441, 1725458118.86668)
Similar Pair: ('1iz6wa', '1kqzkkp'), Times: (1725458819.170641, 1725459032.194739)
Similar Pair: ('gw0oup', '1f7ty2e'), Times: (1725459284.024981, 1725459300.359116)
Similar Pair: ('1tbfv6z', '1wjkezq'), Times: (1725459466.444991, 1725459481.934259)
Similar Pair: ('1wjkezq', '357uj3'), Times: (1725459681.474773, 1725459807.711968)
Similar Pair: ('lg1yl5', '357uj3'), Times: (1725459817.415308, 1725459882.773112)
Similar Pair: ('357uj3', 'ni6bt2'), Times: (1725459882.773112, 1725459910.167222)
Similar Pair: ('tp6l1q', '1hoeid6'), Times: (1725460236.017942, 1725460236.021967)
Similar Pair: ('1p6tzse', 'nnlnhw'), Times: (1725460237.075646, 1725460237.118539)
Similar Pair: ('4b3jqa', '1kqzkkp'), Times: (1725460262.677974, 1725460263.93871)
Similar Pair: ('4b3jqa', '1kqzkkp'), Times: (1725460463.669533, 1725460463.826768)
Similar P

Vzhledem k tomu, že se jedná o dvojici, výsledkem je větší počet detekovaných spojů, v případě větší sekvence bude situace jednodušší

Další postup resp. navrhovaný vývoj metody


*   Rozšíření na libovolnou velikost sekvence (3,4,5). Otázkou je zda to omezit seshora...
*   Hledání podobné sekvence (například **2iruzp gw0oup 1kqzkkp 1vllswt** a podobná **1wllswt 2iruzp 18vhdin drdkli**)
* Využití existujícch knihoven pro práci s textem a jeho analýzu



## Hledání podobných subřetězců

Některé z následujících knihoven mohou být využity k hledání podobných frází, i když s různými úrovněmi přesnosti a složitosti. Zde je přehled i s postupem jak lze podobné fráze najít:


### **difflib (základní metoda)**

*difflib* dokáže porovnávat podobnost mezi dvěma texty, včetně frází.

Ukázka:


```
import difflib

phrases = ["Tento text obsahuje slova", "Tento text je odlišný", "Tento text obsahuje jiná slova"]

similar_phrases = difflib.get_close_matches("Tento text obsahuje slova", phrases, n=2, cutoff=0.6)

print(similar_phrases)

```

### **fuzzywuzzy (vhodné pro porovnání textu)**

Knihovna fuzzywuzzy umožňuje porovnávat podobnost mezi řetězci, a je vhodná i pro hledání podobných frází

Ukázka:



```
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

phrase = "Tento text obsahuje slova"
phrases_list = ["Tento text obsahuje jiná slova", "Text s jiným obsahem", "Tento text je velmi podobný"]

# Najdeme nejpodobnější fráze v seznamu
similar_phrases = process.extract(phrase, phrases_list, scorer=fuzz.ratio)
print(similar_phrases)

```
fuzzywuzzy poskytuje různé metriky (např. fuzz.ratio, fuzz.partial_ratio, fuzz.token_sort_ratio) pro přizpůsobení míry podobnosti mezi frázemi.

### **NLTK (pokročilé zpracování textu)**
Pomocí knihovny NLTK můžete analyzovat a porovnávat podobnosti frází na úrovni slov nebo tokenů. Nástroje jako nltk.edit_distance mohou být použity k měření podobnosti mezi dvěma frázemi na základě počtu úprav (např. vložení, odstranění, nahrazení), které jsou potřebné k transformaci jedné fráze do druhé.

```
import nltk
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import edit_distance

nltk.download('punkt')

phrase1 = "Tento text obsahuje slova"
phrase2 = "Tento text obsahuje jiná slova"
tokens1 = word_tokenize(phrase1)
tokens2 = word_tokenize(phrase2)

distance = edit_distance(tokens1, tokens2)
print(f"Editační vzdálenost mezi frázemi: {distance}")

```





### Hledání podobných sekvencí s pomocí knihovny diff a fuzzywuzzy



In [10]:
from fuzzywuzzy import fuzz
from itertools import combinations

def find_similar_phrases(text, phrase_length, similarity_threshold=80):
    # Rozdělení textu na jednotlivá slova
    words = text.split()

    # Generování všech možných frází s daným počtem slov
    phrases = [' '.join(words[i:i + phrase_length]) for i in range(len(words) - phrase_length + 1)]

    similar_pairs = []

    # Porovnání všech dvojic frází
    for phrase1, phrase2 in combinations(phrases, 2):
        similarity = fuzz.ratio(phrase1, phrase2)  # Porovnání podobnosti
        if similarity >= similarity_threshold:
            similar_pairs.append((phrase1, phrase2, similarity))

    # Seřazení dvojic podle podobnosti sestupně
    similar_pairs = sorted(similar_pairs, key=lambda x: x[2], reverse=True)

    return similar_pairs[:2]

# Použití metody pro nalezení podobných frází o délce 3 slov
similar_phrases = find_similar_phrases(final_string, phrase_length=34)

for phrase1, phrase2, similarity in similar_phrases:
    print(f"Phrase 1: '{phrase1}', Phrase 2: '{phrase2}', Similarity: {similarity}%")


Phrase 1: 'fvdwjp ljs7tb drdkli 1eujf9a mgvi0p drdkli 1eujf9a scmfa8 17elv0d wh5s7c 17elv0d 1pni6gg 357uj3 vj8pux dducrg lg1yl5 dducrg lg1yl5 13ipmgh lg1yl5 d2ry8q lg1yl5 dducrg 649r85 lg1yl5 1s6cs9u lg1yl5 1eujf9a lg1yl5 1oxvk27 lg1yl5 38yath 1vllswt 2iruzp', Phrase 2: 'ljs7tb drdkli 1eujf9a mgvi0p drdkli 1eujf9a scmfa8 17elv0d wh5s7c 17elv0d 1pni6gg 357uj3 vj8pux dducrg lg1yl5 dducrg lg1yl5 13ipmgh lg1yl5 d2ry8q lg1yl5 dducrg 649r85 lg1yl5 1s6cs9u lg1yl5 1eujf9a lg1yl5 1oxvk27 lg1yl5 38yath 1vllswt 2iruzp ogi2do', Similarity: 97%
Phrase 1: 'ljs7tb drdkli 1eujf9a mgvi0p drdkli 1eujf9a scmfa8 17elv0d wh5s7c 17elv0d 1pni6gg 357uj3 vj8pux dducrg lg1yl5 dducrg lg1yl5 13ipmgh lg1yl5 d2ry8q lg1yl5 dducrg 649r85 lg1yl5 1s6cs9u lg1yl5 1eujf9a lg1yl5 1oxvk27 lg1yl5 38yath 1vllswt 2iruzp ogi2do', Phrase 2: 'drdkli 1eujf9a mgvi0p drdkli 1eujf9a scmfa8 17elv0d wh5s7c 17elv0d 1pni6gg 357uj3 vj8pux dducrg lg1yl5 dducrg lg1yl5 13ipmgh lg1yl5 d2ry8q lg1yl5 dducrg 649r85 lg1yl5 1s6cs9u lg1yl5 1eujf9a 

### Modifikace metody pro textovou reprezentaci IP adresyn (pevná velikost 6 znaků)

In [ ]:
import zlib
import base64
def generate_unique_string(ip_address):
    # Calculate CRC32 of the IP address
    crc_value = zlib.crc32(ip_address.encode('utf-8'))
    # Convert the CRC32 value to a Base32-encoded string
    unique_string = base64.b32encode(crc_value.to_bytes(4, byteorder='big')).decode('utf-8')
    # Remove trailing '=' characters
    unique_string = unique_string.rstrip('=')

    return unique_string

print(generate_unique_string('192.168.210.100'))